# SPRINT NUMERO 4 BARONTINI Y LOZADA

### Algo importante a tener en cuenta es que muchas veces le cambiamos el nombre a los df porque tuvimos que hacerlo "en parches", ya que nos corrían los codigos personas con computadoras más potentes. Eso hizo que sea más desorganizado.

## Comenzamos importando e instalando lo necesario 

In [89]:

conda install -c conda-forge lightfm

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [90]:
import numpy as np 
import pandas as pd
from datetime import datetime
from lightfm import LightFM
from scipy.sparse import csr_matrix

## Luego, leemos la infromación necesaria, contenida en formato .CSV

In [91]:
train = pd.read_csv('train.csv')
metadata = pd.read_csv('metadata.csv', sep=';')

## Eliminamos aquellas columnas que no son relevantes y las filas con datos faltantes

In [92]:
train.drop(['customer_id', 'device_type'], axis=1, inplace=True)
metadata.drop(['reduced_title', "episode_title",'description','made_for_tv','close_caption','sex_rating','show_type','violence_rating','pack_premium_2','language_rating','dialog_rating',"cast_first_name", "credits_first_name",'fv_rating','pay_per_view','pack_premium_1','modify_date'], axis=1, inplace=True)

In [93]:
train.dropna(inplace = True)
metadata.dropna(inplace = True)

## Pasamos los datos de fechas y hora a "datetime" y calculamos cuanto tiempo acumulado de casa contenido vieron los usuarios

In [94]:
train["tunein"]=pd.to_datetime(train["tunein"], format= '%Y-%m-%d %H:%M:%S')
train["tuneout"]=pd.to_datetime(train["tuneout"], format= '%Y-%m-%d %H:%M:%S')
train["diferencia"]= train["tuneout"] - train["tunein"]

In [95]:
minutos_vistos=[]
for i in train["diferencia"]:
    u=i.total_seconds()/60
    minutos_vistos.append(u)
train["minutos_vistos"] = minutos_vistos

In [20]:
df_copia=train
clientes = train["account_id"].unique()
columna_minutos_vistos_acumulado=[]

for i in clientes:
    contenidos_cliente = train["asset_id"][train["account_id"] == i]
    columna_valores=[]
    for u in contenidos_cliente:
        a= train["minutos_vistos"][(train["account_id"]==i) & (train["asset_id"] == u)]
        duración_real=np.sum(a)
        columna_minutos_vistos_acumulado.append(duración_real)

df_duración = pd.DataFrame ({"minutos_vistos_acumulados":columna_minutos_vistos_acumulado})
df_copia=df_copia.join(df_duración) 

KeyboardInterrupt: 

In [21]:
df = pd.merge(df_copia[['account_id',"asset_id","minutos_vistos_acumulados","resume"]],
              metadata[['asset_id', 'content_id', "run_time_min","create_date"]], 
              how = 'inner', 
              on = 'asset_id')
df

KeyError: "['minutos_vistos_acumulados'] not in index"

## Como era mucho procesamiento po rel ciclo for anidado dentro de otro ciclo for, pedimos a otra persona que corra el programa, y nos lo envió con el nombre de "df_copia_nuevo_resultados"

In [96]:
df_copia_resultados = pd.read_csv('df_copia_nuevo_resultados.csv')
df_copia_resultados.head(3)

,Unnamed: 0,account_id,asset_id,minutos_vistos,tunein,tuneout,resume,content_id,create_date,run_time_min,start_vod_date,end_vod_date,minutos_vistos_acumulados
0,0,90627,18332.0,43.0,2021-02-18 22:52:00,2021-02-18 23:35:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,43.0
1,1,3389,18332.0,40.0,2021-02-27 16:25:00,2021-02-27 17:05:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,44.0
2,2,3395,18332.0,46.0,2021-02-24 17:24:00,2021-02-24 18:10:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,40.0


## Entonces, aplicamos el siguiente criterio: Si el usuario había visto más del 70% del contenido, lo considerabamos como "vista" y le dabamos un rating igual a 1. 

In [97]:
df_copia_resultados["columna_porcentaje"]=df_copia_resultados["minutos_vistos_acumulados"]/df_copia_resultados["run_time_min"]

In [98]:
columna_gusta=[]
for i in df_copia_resultados["columna_porcentaje"]:
    if i>0.6999:
        columna_gusta.append("1")
    else:
        columna_gusta.append("0")

In [99]:
df_columna_gusta = pd.DataFrame ({"rating":columna_gusta})
df_copia_resultados=df_copia_resultados.join(df_columna_gusta) 
df_copia_resultados.head()

,Unnamed: 0,account_id,asset_id,minutos_vistos,tunein,tuneout,resume,content_id,create_date,run_time_min,start_vod_date,end_vod_date,minutos_vistos_acumulados,columna_porcentaje,rating
0,0,90627,18332.0,43.0,2021-02-18 22:52:00,2021-02-18 23:35:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,43.0,1.000000,1
1,1,3389,18332.0,40.0,2021-02-27 16:25:00,2021-02-27 17:05:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,44.0,1.023256,1
2,2,3395,18332.0,46.0,2021-02-24 17:24:00,2021-02-24 18:10:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,40.0,0.930233,1
3,3,26042,18332.0,48.0,2021-02-24 21:56:00,2021-02-24 22:44:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,49.0,1.139535,1
4,4,26060,18332.0,5.0,2021-02-19 15:10:00,2021-02-19 15:15:00,1,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,4.0,0.093023,0


## Dropeamos los duplicados (ya que acumulamos los minutos vistos) y separamos en los conjuntos de Train y Test. 

In [103]:
df_copia_resultados = df_copia_resultados.drop_duplicates(
  subset = ['account_id', 'content_id'],
  keep = 'last').reset_index(drop = True)

In [104]:
df_copia_resultados["tunein"]=pd.to_datetime(df_copia_resultados["tunein"], format="%Y-%m-%d").dt.date

In [105]:
train = df_copia_resultados[df_copia_resultados.tunein < datetime(2021, 3, 1).date()]

test = df_copia_resultados[df_copia_resultados.tunein >= datetime(2021, 3, 1).date()]

## Armamos la matriz de interacciones 

In [106]:
train.head(3)

,Unnamed: 0,account_id,asset_id,minutos_vistos,tunein,tuneout,resume,content_id,create_date,run_time_min,start_vod_date,end_vod_date,minutos_vistos_acumulados,columna_porcentaje,rating
0,11,3399,18332.0,17.0,2021-02-24,2021-02-24 22:25:00,1,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,15.0,0.348837,0
3,332,107896,18332.0,14.0,2021-02-19,2021-02-19 16:08:00,1,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,29.0,0.674419,0
5,405,89348,18332.0,2.0,2021-02-26,2021-02-26 22:26:00,0,2040.0,2021-02-18T19:05:36.0Z,43.0,2021-02-18T00:00:00.0Z,2021-06-30T23:59:59.0Z,23.0,0.534884,0


In [31]:
train = train.groupby(['account_id', 'content_id'], as_index=False)\
            .agg({'account_id': 'first',
                  'content_id': 'first',
                  'rating': 'mean'
                  })

In [32]:
matriz_interacciones = pd.pivot_table(train[["account_id", "content_id", "rating"]], index='account_id', columns='content_id', values='rating')

In [33]:
matriz_interacciones = matriz_interacciones.fillna(0)

matriz_interacciones.sample(10)

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,...,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0
account_id,,,,,,,,,,,,,,,,,,,,,
92971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13879,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
45845,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95051,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Elaboramos un diccionario para futura referencia de ambas variables

In [107]:
account_id = list(matriz_interacciones.index)
account_dict = {}
counter = 0 
for i in account_id:
    account_dict[i] = counter
    counter += 1

In [108]:
content_id = list(matriz_interacciones.columns)
content_dict = {}
counter = 0 
for i in content_id:
    content_dict[i] = counter
    counter += 1

## Convertimos a csr matrix

In [36]:
matriz_interacciones_csr = csr_matrix(matriz_interacciones.values)
matriz_interacciones_csr

<97505x3687 sparse matrix of type '<class 'numpy.float64'>'
	with 223066 stored elements in Compressed Sparse Row format>

## Entrenamos el modelo

In [109]:
matriz_interacciones.head(3)

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,...,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0
account_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [110]:
model = LightFM(random_state=0,
                loss='warp',
                learning_rate=0.03,
                no_components=100)

model = model.fit(matriz_interacciones_csr,
                  epochs=10,
                  num_threads=16, verbose=False)

## Definimos la función de recomendación

In [111]:
def recomm(model, matriz_interacciones, account_id, account_dict, content_dict, nrec_items = 20, show = True):

    n_accounts, n_contents = matriz_interacciones.shape
    account_x = account_dict[account_id]

    scores = pd.Series(model.predict(account_x,np.arange(n_contents)))
    scores.index = matriz_interacciones.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(matriz_interacciones.loc[account_id,:] \
                                [matriz_interacciones.loc[account_id,:] > 0].index).sort_values(ascending=False))
    
    #Filtro de items que ya leyó el usuario
    scores = [x for x in scores if x not in known_items]
    recomms = scores[0:nrec_items]

    if show == True:
        print ("User: " + str(account_id))
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + str(i))
            counter+=1

        print("\n Recommended Items:")
        
        counter = 1
        for i in recomms:
            print(str(counter) + '- ' + str(i))
            counter+=1
    return recomms

## Realizamos recomendaciones para los cold start, siendo estas los 20 contenidos más populares


In [112]:
cold_start_recomms = train\
                      .groupby("content_id")\
                      .agg({"rating": "mean"})\
                      .sort_values(by="rating", ascending=False)\
                      .head(20)\
                      .index\
                      .to_list()

A todos los usuarios cold start les recomendaremos:

## Ahora, generamos recomendaciones para todos los usuarios.

In [113]:
users = df_copia_resultados.account_id.unique()
non_cold_users = train.account_id.unique()

print(len(users), len(non_cold_users))

113873 97505


In [114]:
recomms_dict = {
    "user": [],
    "recomms": [],
    "cold": []
}

for user in users:
  
  # Recomms personalizadas
  if user in non_cold_users:
    user_recomms = recomm(model, matriz_interacciones, user, account_dict, content_dict, nrec_items = 20, show = False)
    recomms_dict["user"].append(user)
    recomms_dict["recomms"].append(user_recomms)
    recomms_dict["cold"].append(False) #Para luego tener identificados a los usuarios cold
  
  # Recomms cold
  else:
    recomms_dict["user"].append(user)
    recomms_dict["recomms"].append(cold_start_recomms)
    recomms_dict["cold"].append(True) #Para luego tener identificados a los usuarios cold

## Convertimos el diccionario generado en df

In [115]:
recomms_df = pd.DataFrame(recomms_dict)

In [116]:
recomms_df.head()

,user,recomms,cold
0,3399,"[3900.0, 2040.0, 3806.0, 2942.0, 3726.0, 53.0,...",False
1,26153,"[4366.0, 3087.0, 3151.0, 1005.0, 1008.0, 3119....",True
2,3599,"[4366.0, 3087.0, 3151.0, 1005.0, 1008.0, 3119....",True
3,107896,"[2040.0, 3900.0, 604.0, 3806.0, 97.0, 3716.0, ...",False
4,28867,"[53.0, 135.0, 3900.0, 558.0, 1316.0, 657.0, 19...",False


In [118]:
ideal_recoms = test\
    .sort_values('account_id', ascending=False)\
    .groupby('account_id', as_index=False)\
    .agg({'content_id': 'unique'})


## Evaluación con MAP:

In [119]:
df_map = ideal_recoms.merge(recomms_df, how="left", left_on="account_id", right_on="user")[["account_id", "content_id", "recomms"]]

df_map.head()

,account_id,content_id,recomms
0,2,[433.0],"[3806.0, 3900.0, 2040.0, 2178.0, 3292.0, 3381...."
1,3,"[461.0, 3980.0, 4111.0, 2547.0, 3702.0, 3169.0...","[604.0, 3806.0, 2040.0, 2160.0, 3274.0, 292.0,..."
2,4,"[4129.0, 2344.0, 2212.0, 4340.0, 513.0, 565.0,...","[3900.0, 3775.0, 3806.0, 292.0, 2160.0, 3091.0..."
3,5,[2259.0],"[2160.0, 1139.0, 304.0, 20.0, 1529.0, 8.0, 680..."
4,6,"[3902.0, 3388.0, 3211.0, 4065.0, 3386.0, 2900.0]","[1140.0, 3752.0, 3716.0, 2040.0, 2426.0, 3969...."


In [120]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["content_id", "recomms"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [121]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.00901


## 0.00901 es el MAP obtenido